# SGO Credit 10-Year Fiscal Cost Analysis

This notebook calculates the fiscal cost of the Scholarship Granting Organization (SGO) tax credit over the 10-year budget window (2025-2034) relative to the current law baseline without the credit.


In [1]:
# Set up path and imports
import os
import sys

# Set path to Tax-Calculator directory
path = "/Users/haydendublois/Desktop/Modeling Stuff/Tax Calculator/Tax-Calculator"
os.chdir(path)
os.getcwd()


'/Users/haydendublois/Desktop/Modeling Stuff/Tax Calculator/Tax-Calculator'

In [10]:
# Import required libraries
from taxcalc import *
from taxcalc.growfactors import GrowFactors
from taxcalc.records import Records
from taxcalc.policy import Policy
from taxcalc.calculator import Calculator
import pandas as pd
import numpy as np


In [11]:
# Define the reform (SGO credit enabled)
# Format: [single, mjoint, mseparate, headhh, widow]
sgo_reform = {
    "CR_SGO_c": {
        "2025": [1700.0, 3400.0, 1700.0, 1700.0, 1700.0],
        "2026": [1700.0, 3400.0, 1700.0, 1700.0, 1700.0],
        "2027": [1700.0, 3400.0, 1700.0, 1700.0, 1700.0],
        "2028": [1700.0, 3400.0, 1700.0, 1700.0, 1700.0],
        "2029": [1700.0, 3400.0, 1700.0, 1700.0, 1700.0],
        "2030": [1700.0, 3400.0, 1700.0, 1700.0, 1700.0],
        "2031": [1700.0, 3400.0, 1700.0, 1700.0, 1700.0],
        "2032": [1700.0, 3400.0, 1700.0, 1700.0, 1700.0],
        "2033": [1700.0, 3400.0, 1700.0, 1700.0, 1700.0],
        "2034": [1700.0, 3400.0, 1700.0, 1700.0, 1700.0]
    }
}


In [12]:
print("=" * 80)
print("SGO Credit 10-Year Fiscal Cost Analysis")
print("=" * 80)
print("\nBaseline: Current law without SGO credit (CR_SGO_c = 0)")
print("Reform: Current law with SGO credit ($1,700/$3,400)")
print("\nAnalysis period: 2025-2034")
print("=" * 80)


SGO Credit 10-Year Fiscal Cost Analysis

Baseline: Current law without SGO credit (CR_SGO_c = 0)
Reform: Current law with SGO credit ($1,700/$3,400)

Analysis period: 2025-2034


In [13]:
# Set up TMD data directory
tmd_dir = "/Users/haydendublois/Desktop/Modeling Stuff/tax-microdata-benchmarking/tmd/storage/output"

# Create GrowFactors object that uses tmd_growfactors_jason.csv
gf_tmd_path = os.path.join(tmd_dir, "tmd_growfactors_jason.csv")
gf_tmd = GrowFactors(gf_tmd_path)

# Create Records object that uses TMD data, grow factors and weights
print("\nLoading PUF/TMD data...")
recs = Records(
    data=os.path.join(tmd_dir, "tmd_jason.csv.gz"),
    start_year=Records.TMDCSV_YEAR,
    gfactors=gf_tmd,
    weights=os.path.join(tmd_dir, "tmd_weights_jason.csv.gz"),
    adjust_ratios=None
)
print("PUF/TMD data loaded successfully!")



Loading PUF/TMD data...
PUF/TMD data loaded successfully!


In [14]:
# Create baseline and reform policy objects
# Baseline (current law, CR_SGO_c = 0 by default)
pol_base = Policy()

# Reform (with SGO credit)
pol_ref = Policy()
pol_ref.implement_reform(sgo_reform)

# Create calculator objects (will reuse records for each year)
calc_base = Calculator(policy=pol_base, records=recs)
calc_ref = Calculator(policy=pol_ref, records=recs)

# Calculate for each year
results = []
total_cost = 0.0

print("\nCalculating revenues...")
print("-" * 80)
print(f"{'Year':<8} {'Baseline ($B)':<18} {'Reform ($B)':<18} {'Cost ($B)':<18}")
print("-" * 80)

for year in range(2025, 2035):
    # Advance both calculators to the target year
    calc_base.advance_to_year(year)
    calc_ref.advance_to_year(year)
    
    # Calculate taxes
    calc_base.calc_all()
    calc_ref.calc_all()
    
    # Get weighted total revenue (in billions)
    base_rev = calc_base.weighted_total('iitax') / 1e9
    ref_rev = calc_ref.weighted_total('iitax') / 1e9
    
    # Cost = baseline revenue - reform revenue
    cost = base_rev - ref_rev
    total_cost += cost
    
    results.append({
        'year': year,
        'baseline_revenue': base_rev,
        'reform_revenue': ref_rev,
        'cost': cost
    })
    
    print(f"{year:<8} ${base_rev:>15,.2f}    ${ref_rev:>15,.2f}    ${cost:>15,.2f}")



Calculating revenues...
--------------------------------------------------------------------------------
Year     Baseline ($B)      Reform ($B)        Cost ($B)         
--------------------------------------------------------------------------------
2025     $       2,492.18    $       2,220.54    $         271.65
2026     $       2,623.46    $       2,346.92    $         276.55
2027     $       2,732.13    $       2,451.48    $         280.65
2028     $       2,847.01    $       2,562.78    $         284.23
2029     $       3,000.81    $       2,708.07    $         292.74
2030     $       3,169.27    $       2,872.48    $         296.79
2031     $       3,309.59    $       3,008.88    $         300.71
2032     $       3,455.84    $       3,151.20    $         304.65
2033     $       3,604.73    $       3,296.17    $         308.57
2034     $       3,757.50    $       3,445.01    $         312.50


In [15]:
print("-" * 80)
print(f"{'TOTAL':<8} {'':<18} {'':<18} ${total_cost:>15,.2f}")
print("=" * 80)

# Summary
print("\nSummary Statistics:")
print(f"  10-Year Total Cost: ${total_cost:,.2f} billion")
print(f"  Average Annual Cost: ${total_cost/10:,.2f} billion")
print(f"  First Year Cost (2025): ${results[0]['cost']:,.2f} billion")
print(f"  Final Year Cost (2034): ${results[-1]['cost']:,.2f} billion")


--------------------------------------------------------------------------------
TOTAL                                          $       2,929.03

Summary Statistics:
  10-Year Total Cost: $2,929.03 billion
  Average Annual Cost: $292.90 billion
  First Year Cost (2025): $271.65 billion
  Final Year Cost (2034): $312.50 billion


In [16]:
# Save results to CSV
results_df = pd.DataFrame(results)
output_csv = os.path.join(path, 'sgo_credit_cost_analysis.csv')
results_df.to_csv(output_csv, index=False)
print(f"\nResults saved to: {output_csv}")

# Create and save summary report
from datetime import datetime
summary_report = f"""
SGO Credit 10-Year Fiscal Cost Analysis Summary
{'=' * 80}

Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Baseline: Current law without SGO credit (CR_SGO_c = 0)
Reform: Current law with SGO credit ($1,700/$3,400)
Analysis Period: 2025-2034

10-Year Total Cost: ${total_cost:,.2f} billion
Average Annual Cost: ${total_cost/10:,.2f} billion
First Year Cost (2025): ${results[0]['cost']:,.2f} billion
Final Year Cost (2034): ${results[-1]['cost']:,.2f} billion

Annual Breakdown:
{results_df.to_string(index=False)}

Data Source: PUF/TMD (tmd_jason.csv.gz)
"""

# Save summary report
summary_file = os.path.join(path, 'sgo_credit_cost_summary.txt')
with open(summary_file, 'w') as f:
    f.write(summary_report)
print(f"Summary report saved to: {summary_file}")

# Display results dataframe
print("\nResults DataFrame:")
results_df



Results saved to: /Users/haydendublois/Desktop/Modeling Stuff/Tax Calculator/Tax-Calculator/sgo_credit_cost_analysis.csv
Summary report saved to: /Users/haydendublois/Desktop/Modeling Stuff/Tax Calculator/Tax-Calculator/sgo_credit_cost_summary.txt

Results DataFrame:


,year,baseline_revenue,reform_revenue,cost
0,2025,2492.180948,2220.535102,271.645846
1,2026,2623.463241,2346.916717,276.546524
2,2027,2732.130050,2451.480512,280.649538
3,2028,2847.005087,2562.775113,284.229974
4,2029,3000.812642,2708.069170,292.743472
5,2030,3169.273935,2872.484133,296.789802
6,2031,3309.592882,3008.882366,300.710516
7,2032,3455.844028,3151.197768,304.646260
8,2033,3604.734992,3296.169026,308.565966
9,2034,3757.504605,3445.006675,312.497930
